<a href="https://colab.research.google.com/github/AmelxJamal/NLP_Week1_Lab1/blob/main/Amel_Abdelraheem_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal of this lab is to implement a language identifier (LID).

Our first model will be based on Naive Bayes.

In [ ]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

The next function is used to load the data. Each line of the data consist of a label (corresponding to a language), followed by some text, written in that language. Here is an example of data:

```__label__de Zur Namensdeutung gibt es mehrere Varianten.```


In [ ]:
def load_data(filename):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    for line in fin:
        tokens = line.split()
        data.append((tokens[0], tokens[1:]))
    return data

You can now try loading the first dataset `train1.txt` and look what examples look like.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = load_data("/content/drive/MyDrive/NLP_Week1_Lab1/train1.txt")
print(data[0])

('__label__de', ['Ich', 'würde', 'alles', 'tun,', 'um', 'dich', 'zu', 'beschützen.'])


Next, we will start implementing the Naive Bayes method. This technique is based on word counts, and we thus need to start by implementing a function to count the words and labels of our training set.

`n_examples` is the total number of examples

`n_words_per_label` is the total number of words for a given label

`label_counts` is the number of times a given label appears in the training data

`word_counts` is the number of times a word appears with a given label

In [ ]:
def count_words(data):
    n_examples = 0
    n_words_per_label = defaultdict(lambda: 0)
    label_counts = defaultdict(lambda: 0)
    word_counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for example in data:
        label, sentence = example
        ## FILL CODE
        # print(label)
        label_counts[label]+=1
        for word in sentence:
          # word_counts[word]+=1
          n_words_per_label[label] +=1
          word_counts[label][word] +=1

        n_examples+=1

    return {'label_counts': label_counts, 
            'word_counts': word_counts, 
            'n_examples': n_examples, 
            'n_words_per_label': n_words_per_label}

In [ ]:
a = count_words(data)
b = a['n_words_per_label']
b

defaultdict(<function __main__.count_words.<locals>.<lambda>>,
            {'__label__de': 6630,
             '__label__en': 16444,
             '__label__eo': 7647,
             '__label__es': 3927,
             '__label__fr': 4718,
             '__label__hu': 2271,
             '__label__it': 7759,
             '__label__pt': 4044,
             '__label__ru': 7387,
             '__label__tr': 6026})

Next, using the word and label counts from the previous function, we can implement the prediction function.

Here, `mu` is a regularization parameter (Laplace smoothing), and `sentence` is the list of words corresponding to the test example.

In [ ]:
def predict(sentence, mu, label_counts, word_counts, n_examples, n_words_per_label):
    best_label = None
    best_score = float('-inf')

    n_examples +=mu

    for label in word_counts.keys():
        score = 0.0
        ## FILE CODE
        
        # for word in word_counts[label].keys():
        #   word_counts[label][word] += mu

        for word in sentence:
          proba = (mu + word_counts[label][word])/ (len(word_counts[label])*mu +n_words_per_label[label])
          score += np.log(proba)
        
        if score>best_score:
          best_score = score
          best_label = label


    return best_label

The next function will be used to evaluate the Naive Bayes model on a validation set. It computes the accuracy for a particular regularization parameter `mu`.

In [ ]:
def compute_accuracy(valid_data, mu, counts):
    accuracy = 0.0
    for label, sentence in valid_data:
        ## FILL CODE
        label_hat = predict(sentence, mu, counts['label_counts'], counts['word_counts'], counts['n_examples'], counts['n_words_per_label'])
        if label == label_hat:
          accuracy +=1
     
    return (accuracy/len(valid_data)) * 100

In [ ]:
print("")
print("** Naive Bayes **")
print("")

mu = 1.0
train_data = load_data("/content/drive/MyDrive/NLP_Week1_Lab1/train1.txt")
valid_data = load_data("/content/drive/MyDrive/NLP_Week1_Lab1/valid1.txt")
counts = count_words(train_data)

print("Validation accuracy: %.3f" % compute_accuracy(valid_data, mu, counts))
print("")


** Naive Bayes **

Validation accuracy: 91.500

